# ПостНаука

In [1]:
import artm
import hierarchy_utils
import os
import glob
import sys
import numpy as np
import sklearn

/anaconda/lib/python3.6/site-packages/bigartm-0.8.3-py3.6.egg/artm/master_component.py:604: DeprecationWarning: invalid escape sequence \*
/anaconda/lib/python3.6/site-packages/bigartm-0.8.3-py3.6.egg/artm/master_component.py:714: DeprecationWarning: invalid escape sequence \d
/anaconda/lib/python3.6/site-packages/bigartm-0.8.3-py3.6.egg/artm/master_component.py:783: DeprecationWarning: 'async' and 'await' will become reserved keywords in Python 3.7


In [3]:
data_path = '/Users/maria.selezniova/lib/Rysearch/ARTM/Collections/vw/postnauka.txt'
data_path_batches = '/Users/maria.selezniova/lib/Rysearch/ARTM/Collections/pn_batches/'
if len(glob.glob(data_path_batches + "/*.batch")) < 1:
    batch_vectorizer = artm.BatchVectorizer(data_format='vowpal_wabbit', \
                data_path = data_path, batch_size = 225, target_folder=data_path_batches)
else:
    batch_vectorizer = artm.BatchVectorizer(data_path=data_path_batches, data_format='batches')

In [4]:
class_ids={'text':1.0, 'flat_tag':300.0, 'author': 5.0}


In [7]:
# quality measures of models
scores_list = []
"""
scores_list.append(artm.SparsityPhiScore(name='SparsityScoreText',
                                         class_id='ngramm',
                                         #topic_names=topics_subj
                                        ))
scores_list.append(artm.SparsityThetaScore(name='SparsityScoreThetaSubj',
                                           #topic_names=topics_subj
                                          ))

scores_list.append(artm.SparsityPhiScore(name='SparsityScoreAuthor', class_id='author'))
scores_list.append(artm.SparsityPhiScore(name='SparsityScoreTag', class_id='flat_tag'))
scores_list.append(artm.TopicKernelScore(name='TopicKernelText', class_id='text', probability_mass_threshold=0.25))
"""

scores_list.append(artm.TopTokensScore(name='TopTokensScoreText', class_id='text', num_tokens=15))
scores_list.append(artm.TopTokensScore(name='TopTokensTag', class_id='flat_tag', num_tokens=10))
scores_list.append(artm.PerplexityScore(name='PerplexityScore', class_ids=['text']))
scores_list.append(artm.PerplexityScore(name='PerplexityScore_Tag', class_ids=['flat_tag']))

In [9]:
dict_path = '/Users/maria.selezniova/lib/Rysearch/ARTM/Collections/vw/dictionary.txt'
if not os.path.isfile(dict_path):
    dictionary = artm.Dictionary('dictionary')
    dictionary.gather(batch_vectorizer.data_path)
    dictionary.save_text(dict_path)
else:
    dictionary = artm.Dictionary('dictionary')
    dictionary.load_text(dict_path)

### Тут строится 0 уровень

In [10]:
norm_topic_names=['topic_'+str(i) for i in range(0,19)]
background_names=['background_0']
topic_names0=norm_topic_names+background_names


In [11]:
topic_names0[-1:]

['background_0']

In [12]:
regularizers_list = []


#сглаживание фоновой темы по словам
regularizers_list.append(artm.SmoothSparsePhiRegularizer(name='SPPhiTagRegBackground',class_ids=['text'],
                                                         topic_names=background_names,tau=1000)) 
#декоррелирование остальных тем по словам
regularizers_list.append(artm.DecorrelatorPhiRegularizer(name='DecorrPhiReg',class_ids=['text'],
                                                         topic_names=norm_topic_names,
                                                         tau=500000))


"""regularizers_list.append(artm.DecorrelatorPhiRegularizer(name='DecorrPhiTagReg',class_ids=['flat_tag'],tau=500000))
regularizers_list.append(artm.SmoothSparsePhiRegularizer(name='SPPhiTagReg',class_ids=['flat_tag'],
                                                         topic_names=topic_names0[:-1],tau=-15))
regularizers_list.append(artm.SmoothSparsePhiRegularizer(name='SPPhiTextReg',
                                                         class_ids=['text'],
                                                         #topic_names=topics_subj,
                                                         tau=0.01))

#regularizers_list.append(artm.SmoothSparseThetaRegularizer(name='SmoothThetaRegSubj',
                                                           #topic_names=topics_subj,
                                                           tau=0.01))"""

"regularizers_list.append(artm.DecorrelatorPhiRegularizer(name='DecorrPhiTagReg',class_ids=['flat_tag'],tau=500000))\nregularizers_list.append(artm.SmoothSparsePhiRegularizer(name='SPPhiTagReg',class_ids=['flat_tag'],\n                                                         topic_names=topic_names0[:-1],tau=-15))\nregularizers_list.append(artm.SmoothSparsePhiRegularizer(name='SPPhiTextReg',\n                                                         class_ids=['text'],\n                                                         #topic_names=topics_subj,\n                                                         tau=0.01))\n\n#regularizers_list.append(artm.SmoothSparseThetaRegularizer(name='SmoothThetaRegSubj',\n                                                           #topic_names=topics_subj,\n                                                           tau=0.01))"

In [13]:
%%time

topicNum0 = len(topic_names0)

hier = hierarchy_utils.hARTM(class_ids=class_ids,
             regularizers=regularizers_list,
             scores=scores_list, cache_theta=True,
             seed=37)

CPU times: user 13 µs, sys: 0 ns, total: 13 µs
Wall time: 15.7 µs


In [14]:
level0 = artm.ARTM(topic_names=topic_names0, class_ids=class_ids,
             regularizers=regularizers_list,
             scores=scores_list, cache_theta=True,
             seed=37)

In [15]:
%%time

level0.initialize(dictionary=dictionary)
level0.fit_offline(batch_vectorizer, num_collection_passes=1)

CPU times: user 6.46 s, sys: 527 ms, total: 6.98 s
Wall time: 3.94 s


In [16]:
level0.fit_offline(batch_vectorizer, num_collection_passes=10)

In [17]:
theta = level0.get_phi(class_ids=['author'])
theta.shape

(0, 20)

In [18]:
output = ""
tokens = level0.score_tracker["TopTokensTag"].last_tokens
for topic_name in level0.topic_names:
    output += topic_name + ': '
    for word in tokens[topic_name]:    
        output += word + ' '
    output += '\n'
print(output)

topic_0: россия история история_россии философия политическая_философия география античность история_философии аристотель археология 
topic_1: эволюция биология антропология человек палеонтология антропогенез физиология иммунитет происхождение_человека эволюционная_биология 
topic_2: экономика сша религиоведение политика финансовый_кризис финансовый_рынок теория_игр экономическое_развитие демография новая_институциональная_экономика 
topic_3: культура литература массовая_культура культурология философия филология литературоведение кино искусство кинематограф 
topic_4: история христианство ссср политика религия германия власть франция востоковедение европа 
topic_5: астрономия астрофизика звезды космос вселенная галактика земля черные_дыры солнце солнечная_система 
topic_6: язык лингвистика филология русский_язык право юриспруденция урбанистика город римское_право грамматика 
topic_7: социология общество город социология_повседневности дюркгейм_эмиль вебер_макс гоффман_эрвинг социология

In [19]:
output = ""
tokens = level0.score_tracker['TopTokensScoreText'].last_tokens
for topic_name in level0.topic_names:
    output += topic_name + ': '
    for word in tokens[topic_name]:    
        output += word + ' '
    output += '\n'
print(output)

topic_0: россия русский политический философия российский государство война территория земля власть остров понятие место философ находить 
topic_1: вид эволюция группа организм раса эволюционный ген возникать находить отбор жить часть животное антропология период 
topic_2: экономический экономика страна компания цена рынок деньги рост экономист кризис сша решение банк развитие уровень 
topic_3: культура фильм искусство литература кино культурный музей русский смысл автор текст советский массовый герой писать 
topic_4: война политический власть исторический советский революция государство страна историк сталин народ германия имя политика начало 
topic_5: звезда галактика планета земля дыра черный масса солнце объект космический газ телескоп нейтронный вещество солнечный 
topic_6: русский право город словарь речь текст значение языковой часть лингвист глагол предложение форма вид место 
topic_7: социальный социология пространство отношение объект общество город событие социологический де

In [20]:
print(level0.score_tracker['PerplexityScore'].value[-1])
print(level0.score_tracker['PerplexityScore_Tag'].value[-1])

3817.532563531161
60.583170786015316


In [21]:
level0.theta_columns_naming='title'

In [22]:
level0.get_theta()

,pn_1801,pn_1802,pn_1803,pn_1804,pn_1805,pn_1806,pn_1807,pn_1808,pn_1809,pn_1810,...,pn_2466,pn_2467,pn_2468,pn_2469,pn_2470,pn_2471,pn_2472,pn_2473,pn_2474,pn_2475
topic_0,0.000000e+00,1.512103e-11,1.095423e-13,8.715906e-06,2.210971e-15,1.342505e-10,0.000000e+00,9.418347e-10,0.000000e+00,2.060650e-02,...,1.374831e-06,1.888912e-10,1.013159e-05,6.848550e-06,1.691739e-01,2.196766e-06,4.424300e-12,2.241204e-07,6.403310e-12,5.726664e-12
topic_1,4.746096e-16,3.941446e-13,1.718270e-16,1.225071e-02,2.672703e-10,2.000548e-01,0.000000e+00,0.000000e+00,5.211628e-01,1.278224e-08,...,1.907315e-08,1.013613e-10,6.146120e-04,2.849889e-08,1.426192e-11,3.164723e-06,1.595708e-08,5.839699e-09,1.845642e-10,5.317379e-10
topic_2,2.942955e-04,2.675251e-10,4.826166e-16,2.081146e-06,5.509107e-11,3.913971e-01,0.000000e+00,1.421606e-01,0.000000e+00,3.748239e-14,...,2.320503e-09,2.317266e-10,2.351955e-05,3.117344e-01,1.407998e-11,1.561986e-01,6.630672e-11,2.358131e-09,1.823266e-11,1.807044e-11
topic_3,0.000000e+00,1.999160e-08,1.028094e-09,2.068309e-03,9.696851e-15,3.283670e-11,0.000000e+00,0.000000e+00,0.000000e+00,8.976641e-14,...,2.396961e-06,3.811604e-10,3.504179e-06,3.492300e-08,2.267745e-07,1.344331e-05,9.742163e-12,9.997888e-01,8.776477e-13,1.327968e-11
topic_4,0.000000e+00,2.805216e-13,1.413115e-16,1.312547e-05,1.208682e-14,1.274833e-11,0.000000e+00,0.000000e+00,0.000000e+00,1.525962e-13,...,9.999959e-01,6.990820e-11,6.592413e-07,1.971454e-01,5.362427e-01,2.786374e-02,4.572779e-12,2.259315e-08,1.281490e-11,1.077084e-11
topic_5,0.000000e+00,5.940844e-08,8.104951e-16,8.934184e-08,7.930194e-11,3.956776e-11,0.000000e+00,0.000000e+00,0.000000e+00,8.284227e-12,...,1.413612e-09,3.570102e-05,7.016571e-06,1.287302e-08,4.981478e-12,7.064635e-07,1.799425e-11,9.942318e-10,6.559889e-01,1.000000e+00
topic_6,0.000000e+00,9.930072e-01,1.000000e+00,2.730663e-02,5.966295e-13,1.828729e-11,0.000000e+00,0.000000e+00,0.000000e+00,1.024271e-12,...,5.452440e-08,7.301978e-10,8.811677e-01,2.682540e-08,6.372690e-11,8.140738e-06,1.080079e-11,2.106277e-04,1.943837e-11,5.270933e-11
topic_7,4.254494e-16,5.940318e-13,1.578715e-15,7.838140e-05,1.843047e-12,7.179868e-07,6.679595e-01,3.303344e-12,1.784127e-15,4.027145e-01,...,5.068917e-08,8.910106e-09,1.329287e-04,8.634246e-09,1.626297e-10,1.052783e-01,5.412730e-11,4.119932e-09,7.688355e-12,4.821312e-11
topic_8,1.133449e-13,2.407871e-11,3.221655e-16,9.195467e-07,1.473650e-12,4.085474e-01,0.000000e+00,8.578387e-01,0.000000e+00,5.729291e-01,...,1.006264e-07,6.352933e-11,5.367079e-05,4.911128e-01,5.203765e-11,2.671611e-01,2.708063e-11,3.866680e-09,1.680834e-11,3.511023e-11
topic_9,8.496437e-04,4.385163e-12,8.566804e-11,4.859473e-05,5.958865e-01,7.452495e-09,0.000000e+00,0.000000e+00,0.000000e+00,1.469160e-11,...,3.167577e-08,2.016595e-09,2.385520e-05,1.714017e-08,2.232982e-11,3.128201e-05,4.815819e-09,6.993575e-09,2.097508e-11,2.559484e-10


In [23]:
level0.get_phi(class_ids=['text', 'flat_tag'])

,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,background_0
пресса,2.538203e-05,1.154397e-05,7.093475e-05,8.666296e-05,1.049165e-04,5.073374e-15,1.014241e-05,1.440344e-05,2.012433e-05,6.259078e-09,1.401594e-05,2.354605e-11,1.726565e-07,5.638751e-05,1.597834e-04,2.282491e-05,1.463738e-05,3.511052e-05,1.175568e-05,2.313351e-05
проводить,4.335190e-04,6.769288e-04,5.131346e-04,2.993721e-04,7.535883e-04,4.005693e-04,4.879907e-04,5.561120e-04,6.958839e-04,1.138742e-03,1.135693e-03,8.118924e-04,7.513914e-04,9.645792e-04,6.644281e-04,4.973963e-04,3.858914e-04,1.015351e-03,9.242181e-04,2.316340e-05
томление,3.872720e-05,0.000000e+00,0.000000e+00,0.000000e+00,1.233291e-14,4.960453e-10,5.911274e-14,1.872899e-05,4.796238e-07,6.811296e-15,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.312915e-05
защита,2.430673e-04,1.366708e-04,3.659432e-04,6.081562e-05,2.252426e-04,6.707995e-05,1.794638e-04,1.624826e-04,4.335091e-04,2.051114e-04,3.239599e-04,6.060781e-05,6.293752e-04,7.743949e-06,1.929491e-04,6.402541e-06,2.047764e-04,1.988028e-04,1.094806e-05,2.314763e-05
знание,7.638881e-04,4.717166e-04,3.466774e-04,9.203368e-04,3.707010e-04,6.216277e-04,6.234821e-04,8.761744e-04,8.055245e-04,1.662507e-03,4.766417e-04,3.076534e-04,4.625634e-03,4.044739e-04,1.889101e-03,4.785191e-04,8.122463e-04,4.494224e-04,1.587336e-03,2.315034e-05
апериодический,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.133344e-14,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.541254e-09,7.407872e-05,0.000000e+00,0.000000e+00,0.000000e+00,2.556859e-14,0.000000e+00,2.312919e-05
кровь,2.479640e-04,2.882515e-04,2.925443e-05,2.676256e-04,1.736284e-04,1.360680e-06,1.368476e-05,1.071072e-06,7.357039e-05,4.209800e-05,1.989479e-03,1.029184e-05,3.821906e-05,7.287427e-05,1.780396e-04,1.067514e-05,5.055494e-04,7.116367e-05,1.913616e-05,2.314822e-05
распространение,5.141991e-05,4.256530e-04,4.523775e-04,2.757414e-04,3.014009e-04,1.086842e-04,3.304548e-04,1.436166e-04,5.965298e-04,2.001617e-04,1.872537e-04,5.883047e-04,3.090942e-04,1.650935e-04,2.662645e-04,5.582141e-05,2.661012e-05,4.246557e-04,1.034634e-04,2.316201e-05
легковой,0.000000e+00,0.000000e+00,0.000000e+00,1.450389e-07,3.507949e-06,1.125297e-13,8.125350e-06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.833666e-06,0.000000e+00,1.514981e-05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.312915e-05
затрагиваться,1.389419e-12,3.294123e-05,9.541506e-12,0.000000e+00,6.092060e-06,1.254335e-05,0.000000e+00,1.889341e-13,0.000000e+00,1.297189e-05,2.121153e-13,0.000000e+00,2.757218e-05,0.000000e+00,1.450665e-08,6.152732e-13,7.151045e-14,0.000000e+00,1.608243e-09,2.312919e-05


In [24]:
meta_info, phi_ref = level0.master.attach_model(level0.model_pwt)

In [25]:
import pandas as pd
import numpy as np
phi=level0.get_phi(class_ids=['flat_tag'])
phi.loc["математика"]

topic_0         3.731459e-16
topic_1         0.000000e+00
topic_2         3.126915e-06
topic_3         2.915949e-13
topic_4         2.694283e-14
topic_5         0.000000e+00
topic_6         1.177724e-13
topic_7         1.660750e-13
topic_8         2.530423e-16
topic_9         8.231721e-10
topic_10        6.685560e-10
topic_11        5.383225e-06
topic_12        4.355202e-12
topic_13        1.654593e-09
topic_14        5.033002e-13
topic_15        1.585196e-01
topic_16        0.000000e+00
topic_17        1.850447e-08
topic_18        1.918642e-06
background_0    5.622410e-12
Name: математика, dtype: float32

In [26]:
phi_new=level0.get_phi(class_ids=['flat_tag'])
phi_new=np.zeros(phi_new.shape)

In [27]:
phi_new[phi.index.get_loc('математика')] = [1, 0,0,0,0,0,0,0,0,0]
phi_new[phi.index.get_loc('физика')] = [0, 1,0,0,0,0,0,0,0,0]
phi_new[phi.index.get_loc('биология')] = [0, 0,1,0,0,0,0,0,0,0]
phi_new[phi.index.get_loc('технологии')] = [0, 0,0,1,0,0,0,0,0,0]
phi_new[phi.index.get_loc('социология')] = [0, 0,0,0,1,0,0,0,0,0]
phi_new[phi.index.get_loc('философия')] = [0, 0,0,0,0,1,0,0,0,0]
phi_new[phi.index.get_loc('история')] = [0, 0,0,0,0,0,1,0,0,0]
phi_new[phi.index.get_loc('филология')] = [0, 0,0,0,0,0,0,1,0,0]
phi_new[phi.index.get_loc('экономика')] = [0, 0,0,0,0,0,0,0,1,0]
phi_new[phi.index.get_loc('химия')] = [0, 0,0,0,0,0,0,0,0,1]

ValueError: cannot copy sequence with size 10 to array axis with dimension 20

In [ ]:
phi_ref.shape

In [ ]:
phi_new=np.vstack((phi_new, level0.get_phi(class_ids=['text'])))

In [ ]:
phi_new=np.vstack((phi_new, level0.get_phi(class_ids=['author'])))

In [ ]:
for tok in range(phi_ref.shape[0]):
     for top in range(phi_ref.shape[1]):
        phi_ref[tok, top] = phi_new[tok, top]

In [ ]:
level0.get_phi(model_name=level0.model_pwt, class_ids = ['flat_tag']).sum(axis=0)

### Стоп-слова

In [ ]:
tokens_list = []
unique_tokens = []
not_unique = []
stopwords = ''
for topic_name in level0.topic_names:
    unique_tokens = list(set(unique_tokens+tokens[topic_name]))
    tokens_list += tokens[topic_name]
    
counter = {word: 0 for word in unique_tokens}
for word in unique_tokens:
    for topic_name in level0.topic_names:
        if word in tokens[topic_name]:
            counter[word] +=1

for (word, n) in counter.items():
    if (n >= 3):
        stopwords += word + ' '
print(stopwords)

f = open('stopwords_0.txt', 'w')
f.write(stopwords)
f.close()

In [ ]:
hier._levels.append(level0)

In [ ]:
hier._levels

### Тут строится 1 уровень

In [ ]:
hier.del_level(1)

In [ ]:
norm_topic_names=['topic_'+str(i) for i in range(0,19*3)]
background_names=['background_'+str(i) for i in range(0,20)]
topic_names1 = norm_topic_names + background_names

In [ ]:
topic_names1=['topic_'+str(i) for i in range(0,19*4+1)]

In [ ]:
regularizers_list = []

#regularizers_list.append(artm.SmoothSparsePhiRegularizer(name='SPPhiAuthorReg',class_ids=['author'],tau=0.05))
#regularizers_list.append(artm.SmoothSparsePhiRegularizer(name='SPPhiTagReg',class_ids=['flat_tag'],tau=-1))
#regularizers_list.append(artm.DecorrelatorPhiRegularizer(name='DecorrPhiReg',class_ids=['text'],tau=500000000))
regularizers_list.append(artm.DecorrelatorPhiRegularizer(name='DecorrPhiReg1',
                                                #topic_names=norm_topic_names,
                                                         class_ids=['text'],
                                                         tau=5000000))

#regularizers_list.append(artm.DecorrelatorPhiRegularizer(name='DecorrPhiRegB',
#                                                topic_names=background_names,
#                                                         class_ids=['text'],
#                                                         tau=5000000))

regularizers_list.append(artm.SmoothSparsePhiRegularizer(name='SPPhiTextRegB',
                                                topic_names=topic_names1[-1:],
                                                         class_ids=['text'],
                                                         tau=1000))

#regularizers_list.append(artm.SmoothSparseThetaRegularizer(name='SmoothThetaRegSubj',
                                                           #topic_names=topics_subj, tau=100))

In [ ]:
%%time

topicNum1 = (19*4)

level1 = hierarchy_utils.ARTM_Level(level0, phi_batch_weight=10.0**3, 
                                        num_topics=topicNum1,
                                        topic_names=topic_names1,
                                         class_ids=class_ids,
                                          regularizers=regularizers_list,
                                           scores=scores_list,
                                            num_document_passes=1, 
                                             cache_theta=True, seed=124)

In [ ]:
level1.regularizers

### Первые 5 итераций

In [ ]:
%%time

level1.initialize(dictionary = dictionary)
level1.fit_offline(batch_vectorizer, num_collection_passes = 5)

### Изменение регуляризаторов 

In [ ]:
for i in range(len(level0.topic_names)):
    level1.regularizers.add(artm.DecorrelatorPhiRegularizer(name='SPPhiReg_from5it_'+str(i),
                                                topic_names=child_topics[i], 
                                                         tau=200000*len(child_topics[i])), overwrite=True) 
                    
    #level1.regularizers.add(artm.SmoothSparsePhiRegularizer(name='SPPhiReg_from5it_'+str(i), 
    #                                            topic_names=child_topics[i],
    #                                                     tau=-5*len(child_topics[i])**3), overwrite=True) 
    
    
#for k, name in enumerate(level1.topic_names):
#     if (name==child_topics[i][-1]):
#         level1.topic_names[k]='background_'+str(i)

### Следующие 5 итераций

In [111]:
%%time

level1.fit_offline(batch_vectorizer, num_collection_passes = 10)

Wall time: 59.7 s


### Вывод

In [112]:
tokens = level1.score_tracker["TopTokensTag"].last_tokens
output = ''
for topic_name in level1.topic_names:
    output += topic_name + ': '
    for word in tokens[topic_name]:    
        output += word + ' '
    output += '\n'
print(output)

topic_0: биология ген клетка антибиотики органеллы яйцеклетка белок земля продолжительность_жизни нейродегенеративное_заболевание 
topic_1: история_россии политическая_философия античность история_философии археология древний_рим украина вулканизм русская_философия римская_империя 
topic_2: нейрон нейробиология нейропсихология теория_принятия_решений принятие_решений искусственный_интеллект поведенческая_экономика нервная_система поведение_человека синапс 
topic_3: квантовая_физика квантовые_технологии физика квантовая_механика ньютон_исаак фотоны нанофотоника нанотехнологии квантовая_хромодинамика специальная_теория_относительности 
topic_4: наука история_науки маркс_карл гендер академическая_среда социология_права философия_науки библиотека капитализм социология_науки 
topic_5: химия нобелевская_премия молекула гравитационные_волны органическая_химия большой_взрыв темная_энергия углерод полимеры ямр 
topic_6: наука гендер либерализм история_науки социология_науки аналитическая_филосо

In [113]:
print(level1.score_tracker['PerplexityScore'].value[-1])
print(level1.score_tracker['PerplexityScore_Tag'].value[-1])

800.6624849423467
26.28691613285937


### Стоп-слова 1

In [114]:
tokens_list = []
unique_tokens = []
not_unique = []
stopwords = ''
for topic_name in level1.topic_names:
    unique_tokens = list(set(unique_tokens+tokens[topic_name]))
    tokens_list += tokens[topic_name]
    
counter = {word: 0 for word in unique_tokens}
for word in unique_tokens:
    for topic_name in level1.topic_names:
        if word in tokens[topic_name]:
            counter[word] +=1

for (word, n) in counter.items():
    if (n >= 4):
        stopwords += word + ' '
print(stopwords)

f = open('stopwords_1.txt', 'w')
f.write(stopwords)
f.close()

язык вебер_макс астрофизика эволюция медицина человек дюркгейм_эмиль социология_пространства вторая_мировая_война клетка квантовая_физика белки звездное_скопление физиология астрономия галактика солнечная_система грамматика теория_фреймов технологии нанотехнологии земля космос сталин_иосиф юриспруденция молекула стволовые_клетки социология латур_брюно москва полевая_лингвистика яйцеклетка русский_язык ген биология лингвистическая_компаративистика старение римское_право кардиология урбанистика биомедицина городское_пространство днк звезды физика коллайдер экзопланета специальная_теория_относительности черные_дыры история зиммель_георг новое_время палеонтология лингвистика социология_повседневности 


### Подтемы

In [115]:
psi1 = level1.get_psi()

In [116]:
threshold = 0.05
psi1  # p(subtopic|topic)

,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,background_0
topic_0,0.000025,0.032195,0.000067,0.000077,0.000037,0.000030,0.000039,0.000036,0.000097,0.000038,0.037217,0.000030,0.000091,0.000068,0.000077,0.000043,0.001194,0.051064,0.023543,0.039617
topic_1,0.358084,0.001538,0.004693,0.002300,0.005180,0.000251,0.004291,0.001242,0.009960,0.000041,0.000600,0.000034,0.001629,0.000334,0.004123,0.000063,0.003695,0.000069,0.000069,0.002524
topic_2,0.000531,0.003275,0.005096,0.000419,0.000655,0.000038,0.000070,0.000736,0.002590,0.016010,0.001752,0.000125,0.000047,0.000322,0.000634,0.000488,0.001081,0.008675,0.372093,0.000989
topic_3,0.000021,0.000011,0.002107,0.000305,0.000019,0.000973,0.000116,0.000066,0.001161,0.000613,0.000143,0.144352,0.000643,0.002542,0.000054,0.004302,0.000053,0.000066,0.000024,0.000024
topic_4,0.000724,0.000543,0.002122,0.001480,0.000745,0.000776,0.000822,0.010834,0.004424,0.000115,0.001107,0.000507,0.283648,0.000189,0.004991,0.000529,0.000676,0.002089,0.000100,0.000033
topic_5,0.000170,0.000154,0.000747,0.000021,0.000010,0.002592,0.000030,0.000010,0.000025,0.000079,0.005741,0.000446,0.000218,0.270235,0.000168,0.000772,0.000049,0.002049,0.000143,0.000039
topic_6,0.002988,0.000985,0.007779,0.002514,0.001818,0.000438,0.001805,0.000691,0.008288,0.000507,0.000176,0.000331,0.293219,0.000044,0.010047,0.000540,0.000373,0.002993,0.000103,0.000139
topic_7,0.004696,0.000173,0.000379,0.003568,0.189122,0.000041,0.000845,0.000329,0.005034,0.000156,0.000008,0.000018,0.002855,0.000016,0.002907,0.000139,0.001617,0.000079,0.000136,0.000005
topic_8,0.000360,0.000274,0.000045,0.000076,0.000040,0.123587,0.000028,0.000069,0.000165,0.000060,0.000028,0.001350,0.000595,0.009839,0.000023,0.000662,0.000107,0.000056,0.000141,0.000157
topic_9,0.000393,0.000295,0.001271,0.001673,0.000332,0.000057,0.001112,0.159211,0.003827,0.000670,0.000396,0.000020,0.005594,0.000017,0.000203,0.000134,0.000069,0.001331,0.000727,0.001587


In [117]:
tokens0 = level0.score_tracker["TopTokensTag"].last_tokens
tokens1 = level1.score_tracker["TopTokensTag"].last_tokens

In [118]:
child_topics = []
output = ''
related = 0
for t, topic_name in enumerate(level0.topic_names):
    child_topics.append([])
    output += topic_name + ': '
    for word in tokens0[topic_name]:    
        output += word + ' '
    print("\b", output)
    output=''
    for s, topic_name1 in enumerate(level1.topic_names):
        if (psi1.values[s, t] > threshold):
        #&(int(topic_name1[-1])!=3):
            child_topics[t].append(topic_name1)
            related += 1
            output += "    "+ topic_name1 + ': '
            for word in tokens1[topic_name1]:    
                output += word + ' '
            print (output)
            output =''
print("\n edges: ", related)

related_topics = 0
for row in (psi1.values>threshold):
    if (sum(row) != 0):
        related_topics += 1

print("\n child topics: ", related_topics)    

 topic_0: россия история история_россии философия политическая_философия география античность история_философии аристотель археология 
    topic_1: история_россии политическая_философия античность история_философии археология древний_рим украина вулканизм русская_философия римская_империя 
    topic_23: история кавказ политический_режим история_россии история_повседневности шмитт_карл династия ельцин_борис империя ленин 
    topic_27: история_россии политическая_философия древняя_греция античность археология геология петр_i история история_философии кавказ 
    topic_57: история история_повседневности династия кавказ шмитт_карл политический_режим гитлер_адольф империя вторая_мировая_война новое_время 
 topic_1: эволюция биология антропология человек палеонтология антропогенез физиология иммунитет происхождение_человека эволюционная_биология 
    topic_50: эволюция дарвин_чарльз человек эволюционная_биология физиология неандерталец палеонтология происхождение_человека динозавры биолог

In [119]:
tokens0 = level0.score_tracker["TopTokensScoreText"].last_tokens
tokens1 = level1.score_tracker["TopTokensScoreText"].last_tokens

In [109]:
output = ''
related = 0
for t, topic_name in enumerate(level0.topic_names):
    output += topic_name + ': '
    for word in tokens0[topic_name]:    
        output += word + ' '
    print("\b", output)
    output=''
    for s, topic_name1 in enumerate(level1.topic_names):
        if psi1.values[s, t] > threshold:
            related += 1
            output += "    "+ topic_name1 + ': '
            for word in tokens1[topic_name1]:    
                output += word + ' '
            print (output)
            output =''
print("\n edge: ", related)

 topic_0: россия русский политический философия российский государство война территория земля власть остров понятие место философ находить 
    topic_1: россия философия платон аристотель география сократ рим этика архитектура пифагор эрос радиоактивность сейсмология классика бракосочетание 
    topic_23: русь политический власть народ исторический просвещение великий война этнос обряд страна начало имя территория русский 
    topic_27: россия философия география русь аристотель платон рим этика этнография тирания сократ обряд архитектура землетрясение смерть 
 topic_1: вид эволюция группа организм раса эволюционный ген возникать находить отбор жить часть животное антропология период 
    topic_50: антропология неврология стресс иммунитет кооперация депрессия сон homo фауна иммунология раса шизофрения этнос палеозой кроманьонец 
    topic_54: этнография антропология иммунитет иммунология сомнология питание стресс депрессия анатомия этнос вид сон раса австралия изотоп 
    topic_55: а

### Построение третьего уровня

In [ ]:
hier.del_level(2)

In [559]:
hier._levels

[artm.ARTM(num_topics=20, num_tokens=79402, class_ids=['text', 'flat_tag', 'author'])]

In [560]:
regularizers_list = []

#regularizers_list.append(artm.SmoothSparsePhiRegularizer(name='SPPhiAuthorReg',class_ids=['author'],tau=0.05))
#regularizers_list.append(artm.SmoothSparsePhiRegularizer(name='SPPhiTagReg',class_ids=['flat_tag'],tau=-1))
#regularizers_list.append(artm.DecorrelatorPhiRegularizer(name='DecorrPhiReg',class_ids=['text'],tau=500000000))
regularizers_list.append(artm.DecorrelatorPhiRegularizer(name='DecorrPhiReg1',
                                                         topic_names=[
                                                                      'topic_11','topic_12', 'topic_13', 'topic_14', 'topic_15',
                                                                      'topic_16', 'topic_17', 'topic_18', 'topic_19', 'topic_20',
                                                                     'topic_21', 'topic_22', 'topic_23', 'topic_24', 'topic_25',
                                                                      'topic_26', 'topic_27', 'topic_28', 'topic_29',
                                                                     'topic_30', 'topic_31', 'topic_32', 'topic_33', 'topic_34',
                                                                      'topic_35','topic_36', 'topic_37', 'topic_38', 'topic_39',
                                                                      'topic_40', 'topic_41', 'topic_42', 'topic_43', 'topic_44',
                                                                     'topic_45', 'topic_46', 'topic_47', 'topic_48', 'topic_49'],
                                                         tau=50000000))

regularizers_list.append(artm.DecorrelatorPhiRegularizer(name='DecorrPhiReg2',
                                                         topic_names=['topic_0', 'topic_1', 'topic_2', 'topic_3', 'topic_4', 'topic_5',
                                                                     'topic_6', 'topic_7', 'topic_8', 'topic_9', 'topic_10',],
                                                         tau=50000))

regularizers_list.append(artm.SmoothSparsePhiRegularizer(name='SPPhiTextReg',
                                                         class_ids=['text'],
                                                         topic_names=['topic_0', 'topic_1', 'topic_2', 'topic_3', 'topic_4', 'topic_5',
                                                                     'topic_6', 'topic_7', 'topic_8', 'topic_9', 'topic_10',],
                                                         tau=5000))

#regularizers_list.append(artm.SmoothSparseThetaRegularizer(name='SmoothThetaRegSubj',
                                                           #topic_names=topics_subj, tau=100))

In [ ]:
%%time

topicNum2 = 50

level2 = hierarchy_utils.ARTM_Level(level1, phi_batch_weight=10.0**3, 
                                        num_topics=topicNum2,  
                                         class_ids=class_ids,
                                          regularizers=regularizers_list,
                                           scores=scores_list,
                                            num_document_passes=1, 
                                             cache_theta=True, seed=124)

In [ ]:
%%time

level2.initialize(dictionary=dictionary)
level2.fit_offline(batch_vectorizer, num_collection_passes=30)

In [ ]:
psi2 = level2.get_psi()
psi2>0.05

In [ ]:
output = ""
tokens2 = level2.score_tracker["TopTokensTag"].last_tokens
for topic_name in level2.topic_names:
    output += topic_name + ': '
    for word in tokens2[topic_name]:    
        output += word + " "
    output += "\n"
print(output)

In [ ]:
tokens0 = level1.score_tracker["TopTokensTag"].last_tokens
tokens1 = level2.score_tracker["TopTokensTag"].last_tokens

In [ ]:
threshold = 0.05

In [ ]:
output = ''
related = 0
for t, topic_name in enumerate(level1.topic_names):
    output += topic_name + ': '
    for word in tokens0[topic_name]:    
        output += word + ' '
    print("\b", output)
    output=''
    for s, topic_name1 in enumerate(level2.topic_names):
        if psi2.values[s, t] > threshold:
            related += 1
            output += "    "+ topic_name1 + ': '
            for word in tokens1[topic_name1]:    
                output += word + ' '
            print (output)
            output =''
print("\n edges: ", related)

### Сохронение

In [123]:
hier._levels.append(level1)

In [132]:
hier._levels

[artm.ARTM(num_topics=20, num_tokens=45093, class_ids=['text', 'flat_tag', 'author']),
 artm.ARTM(num_topics=77, num_tokens=45093, class_ids=['text', 'flat_tag', 'author'])]

In [133]:
import pickle

In [136]:
%%time

# OUR OWN VERY COOL SAVE

dump = {}

for level_idx, level in enumerate(hier._levels):
    if level_idx > 0:
        # Zeroth level hierarchy doesn't have get_psi()
        dump["psi_%d" % level_idx] = level.get_psi()
    dump["phi_%d" % level_idx] = level.get_phi(class_ids='flat_tag')
dump["theta"] = hier.get_theta()

Wall time: 2.89 s


In [137]:
pickle.dump(dump, open("C:\\Users\\el1318\\Rysearch\\shared\\hartm.mdl", "wb"))

### Grid search

In [ ]:
from sklearn.model_selection import ParameterGrid

In [ ]:
param_grid = {'SPPhiAuthorReg': [0],'SPPhiTagReg': [-5, -2, -0.5, -0.05, 0, 0.05, 0.5, 2, 5], 
             'DecorrPhiReg': [500000000], 'SPPhiTextReg': [-5, -2, -0.5, -0.05, 0, 0.05, 0.5, 2, 5],
             'SmoothThetaReg': [-10, -5, -2, -1, 0, 1, 2, 5, 10] }

In [ ]:
parameters = list(ParameterGrid(param_grid))

In [ ]:
hier.regularizers

In [ ]:
regularizers_names = [regularizers_list[i].name for i in range(0, len(regularizers_list))]
regularizers_list[0]._tau = 1
regularizers_list[0].tau

In [ ]:
%%time

params_textprepl = {}
params_tagsprepl = {}

for taus in parameters:
    for name, value in taus.items():
        for i in range(0, len(regularizers_list)):
            if(regularizers_list[i].name == name):
                regularizers_list[i]._tau = value
    level0.fit_offline(batch_vectorizer, num_collection_passes=10)
    params_textprepl[str(taus)]=level0.score_tracker['PerplexityScore'].value[-1]
    params_tagsprepl[str(taus)]=level0.score_tracker['PerplexityScore_Tag'].value[-1]

In [ ]:
import operator
#print(sorted(params_tagsprepl))
print(sorted(params_textprepl.items(), key=operator.itemgetter(1)))

In [ ]:
print(len(params_textprepl))

In [ ]:
param_grid = {'SPPhiAuthorReg': [0],'SPPhiTagReg': [-0.2, -0.15, -0.1, -0.05, 0, 0.05, 0.1, 0.15, 0.2], 
             'DecorrPhiReg': [500000000], 'SPPhiTextReg': [-0.2, -0.15, -0.1, -0.05, 0, 0.05, 0.1, 0.15, 0.2],
             'SmoothThetaReg': [-0.2, -0.15, -0.1, -0.05, 0, 0.05, 0.1, 0.15, 0.2] }

In [ ]:
%%time

params_textprepl1 = {}
params_tagsprepl1 = {}

for taus in parameters:
    for name, value in taus.items():
        for i in range(0, len(regularizers_list)):
            if(regularizers_list[i].name == name):
                regularizers_list[i]._tau = value
    level0.fit_offline(batch_vectorizer, num_collection_passes=10)
    params_textprepl1[str(taus)]=level0.score_tracker['PerplexityScore'].value[-1]
    params_tagsprepl1[str(taus)]=level0.score_tracker['PerplexityScore_Tag'].value[-1]

In [ ]:
#print(sorted(params_tagsprepl))
print(sorted(params_textprepl.items(), key=operator.itemgetter(1)))

In [ ]:
print(len(params_textprepl1))

In [ ]:
print(sorted(params_tagsprepl1.items(), key=operator.itemgetter(1)))

### Модификация словаря

In [ ]:
#создает словарь, в котором для слов из tokens value=1, а для остальных токенов value=0
def create_my_dict(tokens):
    line='newline'
    with open('E:/lib/ARTM/htest/UCI_collections/UCI_trimmed/dictionary.txt', 'r', encoding='utf-8', errors='replace') as dict0, open('E:/lib/ARTM/htest/UCI_collections/UCI_trimmed/my_dict'+topic_name[-2:]+'.txt', 'w+') as my_dict:
        while (line!=''):
            try:
                line=dict0.readline()
                line=line.split(', ')
                if(len(line)==5):
                    if (line[0] in tokens):
                        line[1] = '1.0'
                    else:
                        line[1] = '0.0'
                line = ', '.join(line)
                my_dict.write(line)
            except UnicodeEncodeError:
                pass
            except UnicodeDecodeError:
                pass
    my_dict = artm.Dictionary('mydict'+topic_name[-2:])
    my_dict.load_text('E:/lib/ARTM/htest/UCI_collections/UCI_trimmed/my_dict'+topic_name[-2:]+'.txt', encoding = 'cp1251')
    print(my_dict)
    return my_dict
    